In [137]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### 대화 내용을 기록하는 Agent

- `RunnableWithMessageHistory` 를 사용하여 인자로 `AgentExecutor` 를 넣어줍니다.

In [138]:
from langchain.agents import tool
from typing import List, Dict, Annotated

In [139]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

In [140]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [141]:
from bs4 import BeautifulSoup
import requests
import re

In [142]:
from langchain_teddynote.messages import AgentStreamParser
from langchain_teddynote.messages import AgentCallbacks
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

### 도구 정의

In [143]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""

    print(f'검색어: {query}')

    news_tool = GoogleNews()

    return news_tool.search_by_keyword(query, k=5)

In [144]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [145]:
@tool
def naver_news_crawl(news_url: str) -> str:                 # 네이버 뉴스기사를 크롤링하여 본문 내용을 리턴하는 도구
    """Crawls a 네이버(naver.com) news article and returns the body content."""

    response = requests.get(news_url)                       # HTTP GET 요청

    if response.status_code == 200:                         # 응답 200. 성공
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h2', id='title_area').get_text()
        content = soup.find('div', id='contents').get_text()    
        
        cleaned_title = re.sub(r'\n{2, }', '\n', title)
        cleaned_content = re.sub(r'\n{2, }', '\n', content)
    else:
        print(f'HTTP 요청 실패. {response.status_code}')
    
    return f'{cleaned_title}\n{cleaned_content}'

### Agent 프롬프트 작성

In [146]:
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news."
            "Be sure to use the 'python_repl_tool' tool when make python code."
            "Be sure to use the 'naver_news_crawl' tool when naver news crawlling."
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

In [147]:
tools = [search_keyword, python_repl_tool, naver_news_crawl]

### Agent 생성

In [148]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)

### AgentExecutor

In [149]:
agent_executor = AgentExecutor(
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=True,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

### 중간 단계를 출력하는 사용자 정의 함수 정의

In [150]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool.get('tool')}")
    print(f'===== 도구 호출 =====')

In [151]:
def obervation_callback(observation) -> None:
    print(f'===== 관찰 내용 =====')
    print(f"Observation : {observation.get('observation')[0]}")
    print(f'===== 관찰 내용 =====')

In [152]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(result)
    print(f'===== 최종 답변 =====')

In [153]:
agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback, 
    observation_callback=obervation_callback, 
    result_callback=result_callback
)

### AgentStreamParser

In [154]:
agent_stream_parser = AgentStreamParser(agent_callbacks)

In [155]:
store = {}

In [156]:
def get_session_history(session_ids):   # session_id 를 기반으로 세션 기록을 가져오는 함수    
    if session_ids not in store:        # session_id 가 store에 없는 경우        
        store[session_ids] = ChatMessageHistory()   # 새로운 ChatMessageHistory 객체를 딕셔너리에 저장
    
    return store[session_ids]           # 해당 세션 ID에 대한 세션 기록 리턴

In [157]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,                                    # session_id 를 기반으로 세션 기록을 가져오는 함수
    input_messages_key='input',                             # 프롬프트의 질문으로 들어가는 key: input
    history_messages_key='chat_history'                     # 프롬프트의 대화 기록이 들어가는 key: chat_history
)

In [158]:
response = agent_with_chat_history.stream(
    {'input': '안녕? 내 이름은 이인환이야.'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [159]:
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
안녕하세요, 이인환님! 어떻게 도와드릴까요?

> Finished chain.
===== 최종 답변 =====
안녕하세요, 이인환님! 어떻게 도와드릴까요?
===== 최종 답변 =====


In [160]:
response = agent_with_chat_history.stream(
    {'input': '내 이름이 뭐라고?'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [161]:
response = agent_with_chat_history.stream(
    {'input': '나는 소프트웨어놀이터 라는 회사에 코딩강의 및 연구 개발을 하고있어'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [162]:
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
소프트웨어놀이터에서 코딩 강의와 연구 개발을 하고 계시군요! 정말 흥미로운 일입니다. 어떤 주제에 대해 강의하시거나 연구하고 계신가요?

> Finished chain.
===== 최종 답변 =====
소프트웨어놀이터에서 코딩 강의와 연구 개발을 하고 계시군요! 정말 흥미로운 일입니다. 어떤 주제에 대해 강의하시거나 연구하고 계신가요?
===== 최종 답변 =====


In [163]:
response = agent_with_chat_history.stream(
    {
        'input': '랭체인에 대한 뉴스 5개를 검색해서 이메일 본문으로 작성해줘.'
        '수신인에는 `윤정 대리님` 그리고, 발신인에는 내 인적정보를 적어줘.'
        '정중한 어조로 작성하고, 메일의 시작과 끝에는 적절한 인사말과 맺음말을 적어줘.'
    }, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [164]:
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : search_keyword
===== 도구 호출 =====

Invoking: `search_keyword` with `{'query': '랭체인'}`


검색어: 랭체인
[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFBtcm1OMmpJOUxvdnlkd0M0dVFfaGhma3BrTDRaVEY2OXFwd2tfUTZHVkFNSkZJYkZLc0x2dzFWWDNldF9TaVhIU0NxSUxyMlpIRC1KNksyLWdNQQ?oc=5', 'content': '개발자를 위한 랭체인 기초 개념과 환경 설정법 - 요즘IT'}, {'url': 'https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5', 'content': '랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS - Samsung SDS'}, {'url': 'https://news.google.com/rss/articles/CBMiRkFVX3lxTE1JSVJ3QkJSODJJeFcwVzJCZXJIcUdsTjg4WWd2dzZkY3Y2dmZuX1g1bEJfX3d4LTRPam1CWXM5bjk5R0xxTmc?oc=5', 'content': '책7- OpenAI API, 랭체인 완벽 활용법 - 브런치'}, {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE5SRG5pSjVLS0ZfZGZHQjN4Yk5jX0VqLWFCWFpDajdSRzhWTXNMZndzV09NWnkwMmlpaVRZWU14Sjl

In [165]:
response = agent_with_chat_history.stream(
    {'input': '내 이름이 뭐라고?'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

In [166]:
for step in response:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
당신의 이름은 이인환입니다.

> Finished chain.
===== 최종 답변 =====
당신의 이름은 이인환입니다.
===== 최종 답변 =====


In [167]:
response = agent_with_chat_history.stream(
    {'input': '좀전에 질문한 내용에 대해서 알려줘'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

for step in response:
    print('step:', step)
    print()
    print('==' * 50)

    if 'actions' in step:
        print(f'[actions]')

        for action in step['actions']:
            print(f"사용 도구 이름 : {action.tool}")
            print(f"입력 쿼리 : {action.tool_input}")
            print(f"Log: {action.log}")

    elif 'steps' in step:
        print(f'[steps]')

        for agent_step in step['steps']:
            # print("Action:")
            print(f"사용 도구 이름 : {agent_step.action.tool}")
            print(f"입력 쿼리 : {agent_step.action.tool_input}")
            print(f"  Log: {agent_step.action.log}")
            print(f"Observation: {agent_step.observation}")


    elif 'output' in step:
        print(f"[output]: {step['output']}")

        if 'messages' in step:
            for message in step['messages']:
                print(f"Message: {message.content}")



> Entering new None chain...
step: {'actions': [ToolAgentAction(tool='search_keyword', tool_input={'query': '랭체인'}, log="\nInvoking: `search_keyword` with `{'query': '랭체인'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Qd7BsNMkVvN2OOpJXjAuuOol', 'function': {'arguments': '{"query":"랭체인"}', 'name': 'search_keyword'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c'}, id='run-2d674627-6c4d-4c4d-9c83-223dcf167896', tool_calls=[{'name': 'search_keyword', 'args': {'query': '랭체인'}, 'id': 'call_Qd7BsNMkVvN2OOpJXjAuuOol', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'search_keyword', 'args': '{"query":"랭체인"}', 'id': 'call_Qd7BsNMkVvN2OOpJXjAuuOol', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_Qd7BsNMkVvN2OOpJXjAuuOol')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'i

In [168]:
response = agent_with_chat_history.stream(
    {'input': '윤정 대리님께 보내기 위해 작성한 이메일 내용을 알려줘'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

for step in response:
    print('step:', step)
    print()
    print('==' * 50)

    if 'actions' in step:
        print(f'[actions]')

        for action in step['actions']:
            print(f"사용 도구 이름 : {action.tool}")
            print(f"입력 쿼리 : {action.tool_input}")
            print(f"Log: {action.log}")

    elif 'steps' in step:
        print(f'[steps]')

        for agent_step in step['steps']:
            # print("Action:")
            print(f"사용 도구 이름 : {agent_step.action.tool}")
            print(f"입력 쿼리 : {agent_step.action.tool_input}")
            print(f"  Log: {agent_step.action.log}")
            print(f"Observation: {agent_step.observation}")


    elif 'output' in step:
        print(f"[output]: {step['output']}")

        if 'messages' in step:
            for message in step['messages']:
                print(f"Message: {message.content}")  



> Entering new None chain...
아래는 윤정 대리님께 보내기 위해 작성한 이메일 내용입니다.

---

**수신인:** 윤정 대리님  
**발신인:** 이인환

---

안녕하세요, 윤정 대리님.

최근 랭체인에 대한 흥미로운 뉴스들을 찾아보았습니다. 아래에 관련된 기사들을 정리해 드립니다.

1. [개발자를 위한 랭체인 기초 개념과 환경 설정법 - 요즘IT](https://news.google.com/rss/articles/CBMiWkFVX3lxTFBtcm1OMmpJOUxvdnlkd0M0dVFfaGhma3BrTDRaVEY2OXFwd2tfUTZHVkFNSkZJYkZLc0x2dzFWWDNldF9TaVhIU0NxSUxyMlpIRC1KNksyLWdNQQ?oc=5)
   
2. [랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS - Samsung SDS](https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5)

3. [책7- OpenAI API, 랭체인 완벽 활용법 - 브런치](https://news.google.com/rss/articles/CBMiRkFVX3lxTE1JSVJ3QkJSODJJeFcwVzJCZXJIcUdsTjg4WWd2dzZkY3Y2dmZuX1g1bEJfX3d4LTRPam1CWXM5bjk5R0xxTmc?oc=5)

4. [주목해야 할 5대 기술 전망 “AI·랭체인·클라우드 전문가·SLM·데이터 전문가” - GTT KOREA](https://news.google.com/rss/articles/CBMiakFVX3lxTE5SRG5pSjVLS0ZfZGZHQjN4Yk5jX0VqLWFCWFpDajdSRzhWTXNMZndzV09